<a href="https://colab.research.google.com/github/anujavenkatachalam04/chvi_vbd_rj/blob/main/notebooks/create_10kmx10km_grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create 10kmx10km Grid/0.1 degree
1. Load 2011 Census shapefile
2. Create 0.1 degree grid
3. Export to geojson

In [1]:
# # import dependencies
# !pip install -r https://raw.githubusercontent.com/anujavenkatachalam04/chvi_vbd_rj/main/requirements.txt

ERROR: Could not install packages due to an OSError: HTTPSConnectionPool(host='pypi.esri.com', port=443): Max retries exceeded with url: /simple/arcgis/ (Caused by NewConnectionError('<pip._vendor.urllib3.connection.HTTPSConnection object at 0x7addc8fe7cb0>: Failed to establish a new connection: [Errno -2] Name or service not known'))



In [2]:
import os
import pandas as pd
import geopandas as gpd
import requests
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')
import geopandas as gpd
from shapely.geometry import Point
import uuid

Mounted at /content/drive


In [3]:
os.chdir("/content/drive/MyDrive/CHVI")

In [17]:
# Import shapefile
blocks_shp=gpd.read_file("5_Shapefiles/SUBDISTRICT_11/Rajasthan_Blocks.shp")

In [18]:
# Check that crs is WGS84, 244 blocks
blocks_shp.crs, len(blocks_shp)

(<Geographic 2D CRS: EPSG:4326>
 Name: WGS 84
 Axis Info [ellipsoidal]:
 - Lat[north]: Geodetic latitude (degree)
 - Lon[east]: Geodetic longitude (degree)
 Area of Use:
 - name: World.
 - bounds: (-180.0, -90.0, 180.0, 90.0)
 Datum: World Geodetic System 1984 ensemble
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich,
 244)

In [20]:
# blocks_shp.plot(color="lightgrey", edgecolor="black")

#Climate - Create ERA5 Land 0.1 degree grid

In [28]:
def create_era5_1deg_grid(blocks_gdf, grid_size_deg=0.1):
    """
    Create a 0.25 degree x 0.25 degree point grid over the given blocks GeoDataFrame
    and attach block attributes (NAME, DISTRICT, C_CODE11).

    The input blocks_gdf MUST be in a geographic CRS (e.g., EPSG:4326/WGS84).

    Args:
        blocks_gdf (GeoDataFrame): Block polygons in a geographic CRS (degrees)
        grid_size_deg (float): Grid spacing in decimal degrees (default: 0.25)

    Returns:
        GeoDataFrame: Grid points with block attributes
    """
    # --- ⚠️ IMPORTANT: Ensure blocks_gdf is in a geographic CRS (like EPSG:4326) ⚠️ ---
   # ERA5 uses degrees (WGS84).
    # If blocks_gdf is not in degrees, you must reproject it first:
    # blocks_gdf = blocks_gdf.to_crs(epsg=4326)

    # Get bounds
    # Geographic CRS uses Longitude (X) and Latitude (Y)
    minx, miny, maxx, maxy = blocks_gdf.total_bounds

    # Create grid coordinates (Longitude and Latitude)
    # Use max + grid_size_deg to ensure the upper bound is inclusive
    x_coords = np.arange(minx, maxx + grid_size_deg, grid_size_deg) # Longitude
    y_coords = np.arange(miny, maxy + grid_size_deg, grid_size_deg) # Latitude

    # Create points (Longitude, Latitude)
    points = [Point(x, y) for x in x_coords for y in y_coords]

    # Use the CRS of the input GeoDataFrame (must be geographic, e.g., EPSG:4326)
    grid = gpd.GeoDataFrame(geometry=points, crs=blocks_gdf.crs)

    # Clip points to blocks union (only keep points inside any block)
    # Use the unary_union for an efficient check against the entire area
    grid_clipped = grid[grid.intersects(blocks_gdf.union_all())].copy()

    # Spatial join to attach block attributes
    grid_with_attrs = gpd.sjoin(
        grid_clipped,
        blocks_gdf[["NAME", "DISTRICT", "C_CODE11", "geometry"]],
        predicate="within",
        how="left"
    )

    # Drop duplicate geometry column added by sjoin
    grid_with_attrs = grid_with_attrs.drop(columns=["index_right"])

    return grid_with_attrs

In [30]:
# create ERA5 grid
grid_1deg = create_era5_1deg_grid(blocks_shp)

In [31]:
# 491 grids
len(grid_1deg)

3095

In [32]:
grid_1deg["C_CODE11"].nunique()

244

In [33]:
# ALL BLOCKS COVERED

244

In [34]:
grid_1deg["grid_id"] = grid_1deg["grid_id"] = grid_1deg.index + 1

In [35]:
grid_1deg.to_file("5_Shapefiles/Rajasthan_1deg.geojson", driver="GeoJSON")

In [ ]:
# The End!